In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2
import subprocess

In [2]:
# Create a new client instance
client = carla.Client('localhost', 2000)
world = client.get_world()

bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = bp_lib.find('vehicle.dodge.charger')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

CarlaSetup = subprocess.Popen(['python', 'carla_setup.py'])
# print("Carla world is running in background!")
# time.sleep(5)

In [3]:
# Start the traffic light controller in the background
traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
print("Traffic light control is running in the background!")

# Start the traffic script in the background
traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '30', '--number-of-walkers', '100'])

print("Traffic generation is running in the background!")

time.sleep(5)

# Set up traffic manager
traffic_manager = client.get_trafficmanager()
vehicle.set_autopilot(True, traffic_manager.get_port())

Traffic light control is running in the background!
Traffic generation is running in the background!


In [4]:
# Cameras Call
# camera_init_trans = carla.Transform(carla.Location(z=1.6, x=0.1))
# Correct sensor placement relative to the vehicle
camera_init_trans = carla.Transform(carla.Location(x=0.0, y=0.0, z=1.6), carla.Rotation(pitch=0.0))



# # Set Camera Attributes Before Spawning
# def set_camera_attributes(camera_bp):
#     camera_bp.set_attribute('image_size_x', '800')
#     camera_bp.set_attribute('image_size_y', '600')
#     camera_bp.set_attribute('fov', '90')

def set_camera_attributes(camera_bp):
    if "optical_flow" in camera_bp.id:
        camera_bp.set_attribute('image_size_x', '1280')  # High resolution
        camera_bp.set_attribute('image_size_y', '720')
        camera_bp.set_attribute('fov', '120')
        camera_bp.set_attribute('sensor_tick', '0.05')  # Capture every 0.05s
        print("✅ Optical Flow camera settings updated!")

    elif "dvs" in camera_bp.id:
        camera_bp.set_attribute('image_size_x', '1280')
        camera_bp.set_attribute('image_size_y', '720')
        camera_bp.set_attribute('fov', '120')
        camera_bp.set_attribute('sensor_tick', '0.01')  # Higher frequency
        camera_bp.set_attribute('positive_threshold', '0.2')  # Lower threshold to detect small brightness changes
        camera_bp.set_attribute('negative_threshold', '0.2')
        camera_bp.set_attribute('refractory_period_ns', '100000')  # Short refractory period (100µs)
        print("✅ DVS camera settings updated!")

    else:  # Default settings for all other cameras
        camera_bp.set_attribute('image_size_x', '800')
        camera_bp.set_attribute('image_size_y', '600')
        camera_bp.set_attribute('fov', '90')  # Default settings
        
# RGB camera
camera_bp = bp_lib.find('sensor.camera.rgb')
set_camera_attributes(camera_bp)
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# Semantic camera
sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation')
set_camera_attributes(sem_camera_bp)
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)

# Instance Seg camera
inst_camera_bp = bp_lib.find('sensor.camera.instance_segmentation')
set_camera_attributes(inst_camera_bp)
inst_camera = world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to=vehicle)

# Monocular Depth camera
depth_camera_bp = bp_lib.find('sensor.camera.depth')
set_camera_attributes(depth_camera_bp)
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle)

# DVS camera
dvs_camera_bp = bp_lib.find('sensor.camera.dvs')
set_camera_attributes(dvs_camera_bp)
dvs_camera = world.spawn_actor(dvs_camera_bp, camera_init_trans, attach_to=vehicle)

# Optical Flow camera
opt_camera_bp = bp_lib.find('sensor.camera.optical_flow')
set_camera_attributes(opt_camera_bp)
opt_camera = world.spawn_actor(opt_camera_bp, camera_init_trans, attach_to=vehicle)

✅ DVS camera settings updated!
✅ Optical Flow camera settings updated!


In [5]:
# Check if DVS Camera is attached correctly
if dvs_camera:
    dvs_parent = dvs_camera.parent
    if dvs_parent:
        print(f"✅ DVS Camera is correctly attached to: {dvs_parent.type_id}")
    else:
        print("❌ DVS Camera is not attached to any actor!")

# Check if Optical Flow Camera is attached correctly
if opt_camera:
    opt_parent = opt_camera.parent
    if opt_parent:
        print(f"✅ Optical Flow Camera is correctly attached to: {opt_parent.type_id}")
    else:
        print("❌ Optical Flow Camera is not attached to any actor!")

✅ DVS Camera is correctly attached to: vehicle.dodge.charger
✅ Optical Flow Camera is correctly attached to: vehicle.dodge.charger


In [6]:
if dvs_camera is None:
    print("❌ Error: DVS Camera failed to spawn!")
    exit()  # Stop execution if the sensor fails to spawn
else:
    print(f"✅ DVS Camera spawned at {dvs_camera.get_transform()}")

if opt_camera is None:
    print("❌ Error: Optical Flow Camera failed to spawn!")
    exit()  # Stop execution if the sensor fails to spawn
else:
    print(f"✅ Optical Flow Camera spawned at {opt_camera.get_transform()}")

if camera is None:
    print("❌ Error: RGB Camera failed to spawn!")
else:
    print(f"✅ RGB Camera spawned at {camera.get_transform()}")

if sem_camera is None:
    print("❌ Error: Sem Camera failed to spawn!")
else:
    print(f"✅ Sem Camera spawned at {sem_camera.get_transform()}")

if inst_camera is None:
    print("❌ Error: Inst Camera failed to spawn!")
else:
    print(f"✅ Inst Camera spawned at {inst_camera.get_transform()}")

if depth_camera is None:
    print("❌ Error: Depth Camera failed to spawn!")
else:
    print(f"✅ Depth Camera spawned at {depth_camera.get_transform()}")


✅ DVS Camera spawned at Transform(Location(x=-15.124249, y=69.708885, z=1.488181), Rotation(pitch=0.458488, yaw=0.066119, roll=-0.194225))
✅ Optical Flow Camera spawned at Transform(Location(x=-15.124249, y=69.708885, z=1.488181), Rotation(pitch=0.458488, yaw=0.066119, roll=-0.194225))
✅ RGB Camera spawned at Transform(Location(x=-15.124249, y=69.708885, z=1.488181), Rotation(pitch=0.458488, yaw=0.066119, roll=-0.194225))
✅ Sem Camera spawned at Transform(Location(x=-15.124249, y=69.708885, z=1.488181), Rotation(pitch=0.458488, yaw=0.066119, roll=-0.194225))
✅ Inst Camera spawned at Transform(Location(x=-15.124249, y=69.708885, z=1.488181), Rotation(pitch=0.458488, yaw=0.066119, roll=-0.194225))
✅ Depth Camera spawned at Transform(Location(x=-15.124249, y=69.708885, z=1.488181), Rotation(pitch=0.458488, yaw=0.066119, roll=-0.194225))


In [7]:
# # Get image dimensions
# image_w = camera_bp.get_attribute('image_size_x').as_int()
# image_h = camera_bp.get_attribute('image_size_y').as_int()

# # Initialize sensor data storage
# sensor_data = {
#     'rgb_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
#     'sem_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
#     'depth_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
#     'dvs_image': np.zeros((image_h, image_w, 3), dtype=np.uint8),  # DVS doesn't need 4 channels
#     'opt_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
#     'inst_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
# }

# Initialize sensor data storage BEFORE calling `listen()`
image_w = 800
image_h = 600

sensor_data = {
    'rgb_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'sem_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'depth_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'dvs_image': np.zeros((image_h, image_w, 3), dtype=np.uint8),  # DVS uses 3 channels
    'opt_image': np.zeros((image_h, image_w, 3), dtype=np.uint8),  # Optical Flow uses 3 channels
    'inst_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
}

In [8]:
def add_label(image, label, position=(10, 50), font_scale=0.8, color=(255, 255, 255)):
    """Adds a text label to an image."""
    labeled_image = image.copy()  # Work on a copy to avoid modifying original
    cv2.putText(labeled_image, label, position, cv2.FONT_HERSHEY_SIMPLEX, 
                font_scale, color, 2, cv2.LINE_AA)
    return labeled_image  # Return the labeled image

def rgb_callback(image, data_dict):
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    add_label(img, "RGB Camera")
    data_dict['rgb_image'] = img

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    add_label(img, "Semantic Segmentation")
    data_dict['sem_image'] = img

def inst_callback(image, data_dict):
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    add_label(img, "Instance Segmentation")
    data_dict['inst_image'] = img

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    add_label(img, "Depth Camera")
    data_dict['depth_image'] = img

# def opt_callback(image, data_dict):
#     img = image.get_color_coded_flow()
#     img = np.array(img, dtype=np.uint8)  # Convert to numpy array
#     # Ensure Optical Flow visualization is not completely black
#     if img.max() == 0:
#         img[:, :, :] = 255  # Set white to indicate no data
#     else:
#         add_label(img, "Optical Flow")
#     data_dict['opt_image'] = img

 
# def dvs_callback(image, data_dict):
#     dvs_events = np.frombuffer(image.raw_data, dtype=np.dtype([
#         ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', np.bool_)]))

#     dvs_img = np.zeros((image.height, image.width, 3), dtype=np.uint8)

#     if len(dvs_events) > 0:  # Ensure there are DVS events
#         dvs_img[dvs_events['y'], dvs_events['x'], dvs_events['pol'] * 2] = 255
#         add_label(dvs_img, "DVS Camera")
#     else:
#         dvs_img[:, :, :] = 255  # Set white if no events detected

#     data_dict['dvs_image'] = dvs_img


In [9]:
def opt_callback(image, data_dict):
    try:
        print(f"📷 Optical Flow Image Received: {image.frame}")

        # Convert flow to color image
        img = image.get_color_coded_flow()
        img = np.array(img, dtype=np.uint8)

        # Rescale image to better visualize motion
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

        print(f"🔍 Optical Flow - Max: {img.max()}, Min: {img.min()}, Mean: {np.mean(img)}")

        if img.max() == 0:
            print("⚠ Warning: Optical Flow is black (no motion detected)")
            img[:, :, :] = 255  # White fallback

        add_label(img, "Optical Flow")
        data_dict['opt_image'] = img

    except Exception as e:
        print(f"❌ Error in Optical Flow processing: {e}")

In [10]:
def dvs_callback(image, data_dict):
    try:
        dvs_events = np.frombuffer(image.raw_data, dtype=np.dtype([
            ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', np.bool_)]))

        dvs_img = np.zeros((image.height, image.width, 3), dtype=np.uint8)

        print(f"📸 DVS Events Detected: {len(dvs_events)}")

        if len(dvs_events) > 0:
            # Map positive events to red, negative events to blue
            dvs_img[dvs_events['y'], dvs_events['x'], 2] = dvs_events['pol'] * 255  # Red for positive
            dvs_img[dvs_events['y'], dvs_events['x'], 0] = (1 - dvs_events['pol']) * 255  # Blue for negative
            add_label(dvs_img, "DVS Camera")
        else:
            print("⚠ Warning: No DVS events detected")
            dvs_img[:, :, :] = 255  # White fallback

        data_dict['dvs_image'] = dvs_img

    except Exception as e:
        print(f"❌ Error in DVS processing: {e}")


In [11]:
def opt_callback(image, data_dict):
    print(f"📷 Optical Flow Image Received: {image.frame}")
    # Process normally

In [12]:
print(f"DVS Image Max: {np.max(sensor_data['dvs_image'])}")
print(f"Optical Flow Image Max: {np.max(sensor_data['opt_image'])}")

DVS Image Max: 0
Optical Flow Image Max: 0


In [13]:
# Start sensor listeners
camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
inst_camera.listen(lambda image: inst_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))
dvs_camera.listen(lambda image: dvs_callback(image, sensor_data))
opt_camera.listen(lambda image: opt_callback(image, sensor_data))

In [ ]:
# Create display windows
cv2.namedWindow('All Cameras', cv2.WINDOW_NORMAL)
cv2.resizeWindow('All Cameras', 1920, 1080)

cv2.namedWindow('Duplicate RGB', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Duplicate RGB', 800, 600)  # Keep RGB window smaller

def preprocess_image(img):
    """Ensure all images have 3 channels (convert to RGB if needed)."""
    if img.shape[2] == 4:  # If image has 4 channels (RGBA)
        img = img[:, :, :3]  # Drop the alpha channel
    return img

while True:
    try:
        # Convert all sensor images to 3-channel RGB format
        sensor_data_processed = {key: preprocess_image(img) for key, img in sensor_data.items()}

        # Check if all images are available before displaying
        if any(img.shape[0] == 0 for img in sensor_data_processed.values()):
            print("Waiting for camera feeds...")
            continue

        # Get the RGB image **without** label for the Duplicate RGB window
        rgb_clean = sensor_data_processed['rgb_image'].copy()

        # Create a **labeled** RGB image for the "All Cameras" window
        rgb_labeled = add_label(sensor_data_processed['rgb_image'], "RGB Camera")

        # Concatenate images in a 2-row format for display
        top_row = np.concatenate([
            rgb_labeled,  # **Labeled RGB Image for "All Cameras"**
            add_label(sensor_data_processed['sem_image'], "Semantic Segmentation"),
            add_label(sensor_data_processed['inst_image'], "Instance Segmentation")
        ], axis=1)
        
        lower_row = np.concatenate([
            add_label(sensor_data_processed['depth_image'], "Depth Camera"),
            add_label(sensor_data_processed['dvs_image'], "DVS Camera"),
            add_label(sensor_data_processed['opt_image'], "Optical Flow")
        ], axis=1)

        # Combine both rows into a single tiled view
        tiled = np.concatenate((top_row, lower_row), axis=0)

        # Display the combined camera feeds in the main window
        cv2.imshow('All Cameras', tiled)

        # Display the **clean RGB image** in the Duplicate RGB window
        cv2.imshow("Duplicate RGB", rgb_clean)

        # Exit on pressing 'q'
        if cv2.waitKey(1) == ord('q'):
            break
    except Exception as e:
        print(f"Error displaying images: {e}")

# Cleanup: Close all OpenCV windows
cv2.destroyAllWindows()

📷 Optical Flow Image Received: 1983
📷 Optical Flow Image Received: 1984
📷 Optical Flow Image Received: 1985
📷 Optical Flow Image Received: 1986
📷 Optical Flow Image Received: 1987
📷 Optical Flow Image Received: 1988
📷 Optical Flow Image Received: 1989
📷 Optical Flow Image Received: 1990
📷 Optical Flow Image Received: 1991
📷 Optical Flow Image Received: 1992
📷 Optical Flow Image Received: 1993
📷 Optical Flow Image Received: 1994
📷 Optical Flow Image Received: 1995
📷 Optical Flow Image Received: 1996
📷 Optical Flow Image Received: 1997
📷 Optical Flow Image Received: 1998
📷 Optical Flow Image Received: 1999
📷 Optical Flow Image Received: 2000
📷 Optical Flow Image Received: 2001
📷 Optical Flow Image Received: 2002
📷 Optical Flow Image Received: 2003
📷 Optical Flow Image Received: 2004
📷 Optical Flow Image Received: 2005


In [ ]:
# camera_process = subprocess.Popen(['python', 'camera_processing.py'])
# print("Camera processing is running in the background!")